In [1]:
import sklearn as skl
import numpy as np
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import pandas as pd
from skimage import filters
import sys
sys.path.append('/Users/kategroschner/Documents/pyNanoFind/utilities/')
import realDataProcess as rdp
from skimage import exposure
from skimage import morphology as morph
from skimage.util import invert
from skimage import feature
import ipywidgets as widgets
from ipywidgets import interact, interactive
from numpy import random

In [2]:
data = h5py.File('/Users/kategroschner/Downloads/Chiral_D_2018_10_31_FilteredStack.emd','r')
subdata = data['data']
subdata = subdata['Masked Particles']
subdata = subdata['data'][:,:,:]

In [3]:
labels = np.loadtxt('/Users/kategroschner/Downloads/Chiral_D_ManualValues_2018_10_31.csv',delimiter=',')

In [4]:
def balance_noFive(images, labels, min_num_labels):
    final_image = []
    final_labels = []
    count_left = 0
    count_right = 0
    count_garbage = 0
    #count_idk = 0
    for idx, l in enumerate(labels):
        if l == 4:
            if count_left < min_num_labels:
                final_image.append(images[idx])
                final_labels.append(l)
                count_left+=1
        if l == 5:
            if count_garbage < min_num_labels:
                final_image.append(images[idx])
                final_labels.append(0)
                count_garbage += 1
        if l == 6:
            if count_right < min_num_labels:
                final_image.append(images[idx])
                final_labels.append(l)
                count_right += 1
        if l == 0:
            if count_garbage < min_num_labels:
                final_image.append(images[idx])
                final_labels.append(l)
                count_garbage += 1
    return np.array(final_image),np.array(final_labels)

In [5]:
sort_images, sort_labels = balance_noFive(subdata,labels,90)

In [9]:
def shuffle(img_stack,labels):
    vals = np.arange(0,len(labels))
    random.shuffle(vals)
    new_stack = np.zeros(img_stack.shape)
    new_labels = np.ones(labels.shape)
    for old_idx,new_idx in enumerate(vals):
        new_stack[old_idx] = img_stack[new_idx]
        new_labels[old_idx] = labels[new_idx]
    return new_stack,new_labels

In [10]:
final_images, final_labels = shuffle(sort_images,sort_labels)

In [11]:
np.save('particles_nofive.npy',final_images)
np.save('labels_nofive.npy',final_labels)

In [12]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [22]:
from keras.preprocessing.image import ImageDataGenerator

In [23]:
from keras.utils import to_categorical

In [17]:
images = final_images[:,50:150,50:150]
images = np.expand_dims(images,axis = 3)

In [27]:
lbls = final_labels.copy()

In [28]:
lbls[lbls == 4] = 1
lbls[lbls == 6] =2

In [31]:
labels = to_categorical(lbls)

In [32]:
labels.shape

(270, 3)

In [58]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [34]:
batch_size = 16

In [35]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [41]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [38]:
trainX = images[:175]
trainY = labels[:175]

In [40]:
valX = images[175:225]
valY = labels[175:225]

In [39]:
seed = 42

In [48]:
train_generator = train_datagen.flow(trainX, y=trainY, batch_size=batch_size,seed=seed)
val_generator = test_datagen.flow(valX,y=valY,batch_size=batch_size,seed=seed)

In [50]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 98, 98, 32)        320       
_________________________________________________________________
activation_12 (Activation)   (None, 98, 98, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 47, 47, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 47, 47, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 21, 21, 64)        18496     
__________

In [59]:
model.fit_generator(
        train_generator,
        steps_per_epoch=500 // batch_size,
        epochs=10,
        validation_data=val_generator,
        validation_steps=200 // batch_size)

Epoch 1/10
31/31 [==============================] - 6s 191ms/step - loss: 1.0120 - acc: 0.4709 - val_loss: 0.7821 - val_acc: 0.6200
Epoch 2/10
31/31 [==============================] - 7s 225ms/step - loss: 0.8687 - acc: 0.5511 - val_loss: 0.7228 - val_acc: 0.6800
Epoch 3/10
31/31 [==============================] - 7s 214ms/step - loss: 0.8750 - acc: 0.5667 - val_loss: 0.6695 - val_acc: 0.7600
Epoch 4/10
31/31 [==============================] - 6s 184ms/step - loss: 0.8517 - acc: 0.6082 - val_loss: 0.6261 - val_acc: 0.7400
Epoch 5/10
31/31 [==============================] - 5s 177ms/step - loss: 0.8642 - acc: 0.5984 - val_loss: 0.7135 - val_acc: 0.6600
Epoch 6/10
31/31 [==============================] - 6s 191ms/step - loss: 0.8465 - acc: 0.6167 - val_loss: 0.6714 - val_acc: 0.6600
Epoch 7/10
31/31 [==============================] - 6s 180ms/step - loss: 0.7741 - acc: 0.6690 - val_loss: 0.5967 - val_acc: 0.7600
Epoch 8/10
31/31 [==============================] - 6s 183ms/step - loss: 0.

In [60]:
predicted = model.predict(images[225:250])

In [62]:
predicted

array([[  0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   1.37820780e-18,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   2.16236117e-25,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  4.43175980e-29,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  4.00618857e-30,   1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  6.88517474e-08,   9.99999881e-01,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.0000000

In [63]:
pred_bin = predicted.copy()
pred_bin[pred_bin<0.95] = 0
pred_bin[pred_bin>0] = 1

In [69]:
p_lbl = []
for i in pred_bin:
    if i[0] == 1:
        p_lbl.append(0)
    elif i[1] == 1:
        p_lbl.append(1)
    elif i[2] == 1:
        p_lbl.append(2)

In [70]:
p_lbl = np.array(p_lbl)

In [71]:
p_lbl.shape

(25,)

In [73]:
metrics.accuracy_score(lbls[225:250],p_lbl)

0.56000000000000005

In [55]:
predicted

array([[  0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  2.27130633e-02,   1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   8.18424800e-04],
       [  1.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   1.65321794e-03,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  9.99733031e-01,   1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   1.61730333e-27,   0.00000000e+00],
       [  2.65318931e-07,   1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   1.0000000

In [74]:
model.save('basic_cnn.h5')

In [76]:
# dimensions of our images.
img_width, img_height = 100, 100

top_model_weights_path = 'bottleneck_fc_model.h5'
epochs = 50
batch_size = 16

In [75]:
from keras import applications

In [83]:
from skimage.color import gray2rgb

In [84]:
trainX_rbg = np.zeros((175,100,100,3))
for idx,img in enumerate(trainX):
    t = img[:,:,0]
    trainX_rbg[idx] = gray2rgb(t)

In [87]:
trainX.shape

(175, 100, 100, 1)

In [89]:
valX_rbg = np.zeros((50,100,100,3))
for idx,img in enumerate(valX):
    t = img[:,:,0]
    valX_rbg[idx] = gray2rgb(t)

In [106]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet', input_shape=(100,100,3))

    generator = datagen.flow(
        trainX_rbg,
        y = trainY,
        batch_size=batch_size,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, len(trainX_rbg) // batch_size)
    np.save('bottleneck_features_train.npy',
            bottleneck_features_train)

    generator = datagen.flow(
        valX_rbg,
        y = valY,
        batch_size=batch_size,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, len(valX_rbg) // batch_size)
    np.save('bottleneck_features_validation.npy',
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array(
        [0] * (len(trainX_rbg) // 2) + [1] * (len(trainX_rbg) // 2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array(
        [0] * (len(valX_rbg) // 2) + [1] * (len(valX_rbg) // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


In [98]:
save_bottlebeck_features()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 0: invalid start byte

In [107]:
train_top_model()

ValueError: Input arrays should have the same number of samples as target arrays. Found 160 input samples and 174 target samples.